# How to Handle Webhooks

> Process Stripe events in your application

This guide shows you how to handle Stripe webhooks to process events like successful payments and subscription changes.

## Problem

You need to respond to events from Stripe (like successful payments) to update your application state, send emails, or fulfill orders.

## Solution

Set up webhook endpoints that Stripe will call when events occur, then use FastStripe to verify and process these events.

## Setting up webhooks in Stripe

1. Go to the [Stripe Dashboard](https://dashboard.stripe.com/webhooks)
2. Click "Add endpoint"
3. Enter your endpoint URL: `https://yoursite.com/stripe/webhook`
4. Select events to listen for:
   - `checkout.session.completed`
   - `payment_intent.succeeded`
   - `customer.subscription.created`
   - `customer.subscription.updated`
   - `customer.subscription.deleted`
5. Copy the webhook signing secret

## Flask webhook handler

In [ ]:
from flask import Flask, request, jsonify
import stripe
import os

app = Flask(__name__)

# Use the official Stripe SDK for webhook verification
stripe.api_key = os.environ['STRIPE_SECRET_KEY']
webhook_secret = os.environ['STRIPE_WEBHOOK_SECRET']

@app.route('/stripe/webhook', methods=['POST'])
def stripe_webhook():
    payload = request.get_data()
    sig_header = request.headers.get('Stripe-Signature')
    
    try:
        # Verify the webhook signature
        event = stripe.Webhook.construct_event(
            payload, sig_header, webhook_secret
        )
    except ValueError:
        # Invalid payload
        return jsonify({'error': 'Invalid payload'}), 400
    except stripe.error.SignatureVerificationError:
        # Invalid signature
        return jsonify({'error': 'Invalid signature'}), 400
    
    # Handle the event
    if event['type'] == 'checkout.session.completed':
        handle_checkout_completed(event['data']['object'])
    elif event['type'] == 'payment_intent.succeeded':
        handle_payment_succeeded(event['data']['object'])
    elif event['type'] == 'customer.subscription.created':
        handle_subscription_created(event['data']['object'])
    elif event['type'] == 'customer.subscription.updated':
        handle_subscription_updated(event['data']['object'])
    elif event['type'] == 'customer.subscription.deleted':
        handle_subscription_cancelled(event['data']['object'])
    else:
        print(f'Unhandled event type: {event["type"]}')
    
    return jsonify({'status': 'success'})

def handle_checkout_completed(checkout_session):
    """Handle successful checkout completion"""
    print(f"Checkout completed: {checkout_session['id']}")
    
    # Get customer information
    customer_id = checkout_session.get('customer')
    customer_email = checkout_session.get('customer_email')
    
    if checkout_session['mode'] == 'payment':
        # One-time payment
        print(f"One-time payment completed for {customer_email}")
        # TODO: Fulfill order, send confirmation email
        
    elif checkout_session['mode'] == 'subscription':
        # Subscription started
        subscription_id = checkout_session.get('subscription')
        print(f"Subscription {subscription_id} started for {customer_email}")
        # TODO: Activate user account, send welcome email

def handle_payment_succeeded(payment_intent):
    """Handle successful payment"""
    print(f"Payment succeeded: {payment_intent['id']}")
    amount = payment_intent['amount'] / 100
    print(f"Amount: ${amount:.2f}")
    # TODO: Update order status, send receipt

def handle_subscription_created(subscription):
    """Handle new subscription"""
    print(f"Subscription created: {subscription['id']}")
    customer_id = subscription['customer']
    # TODO: Set up user access, send welcome materials

def handle_subscription_updated(subscription):
    """Handle subscription changes"""
    print(f"Subscription updated: {subscription['id']}")
    status = subscription['status']
    
    if status == 'active':
        # Subscription is active
        print("Subscription is now active")
        # TODO: Ensure user has access
    elif status == 'past_due':
        # Payment failed
        print("Subscription payment failed")
        # TODO: Send dunning emails, warn user
    elif status == 'canceled':
        # Subscription cancelled
        print("Subscription cancelled")
        # TODO: Revoke access, send cancellation confirmation

def handle_subscription_cancelled(subscription):
    """Handle subscription cancellation"""
    print(f"Subscription cancelled: {subscription['id']}")
    customer_id = subscription['customer']
    # TODO: Revoke user access, send cancellation email

if __name__ == '__main__':
    app.run(debug=True)

## FastAPI webhook handler

In [ ]:
from fastapi import FastAPI, Request, HTTPException
import stripe
import os

app = FastAPI()

stripe.api_key = os.environ['STRIPE_SECRET_KEY']
webhook_secret = os.environ['STRIPE_WEBHOOK_SECRET']

@app.post('/stripe/webhook')
async def stripe_webhook(request: Request):
    payload = await request.body()
    sig_header = request.headers.get('stripe-signature')
    
    try:
        event = stripe.Webhook.construct_event(
            payload, sig_header, webhook_secret
        )
    except ValueError:
        raise HTTPException(status_code=400, detail="Invalid payload")
    except stripe.error.SignatureVerificationError:
        raise HTTPException(status_code=400, detail="Invalid signature")
    
    # Handle events
    event_handlers = {
        'checkout.session.completed': handle_checkout_completed,
        'payment_intent.succeeded': handle_payment_succeeded,
        'customer.subscription.created': handle_subscription_created,
        'customer.subscription.updated': handle_subscription_updated,
        'customer.subscription.deleted': handle_subscription_cancelled,
    }
    
    handler = event_handlers.get(event['type'])
    if handler:
        await handler(event['data']['object'])
    else:
        print(f'Unhandled event type: {event["type"]}')
    
    return {'status': 'success'}

# Event handlers would be the same as in the Flask example

## Testing webhooks locally

Use the Stripe CLI to forward webhooks to your local development server:

In [ ]:
# Install Stripe CLI first: https://stripe.com/docs/stripe-cli

# Login to your Stripe account
# stripe login

# Forward webhooks to your local server
# stripe listen --forward-to localhost:5000/stripe/webhook

# The CLI will show you the webhook signing secret to use

## Using FastStripe with webhooks

You can use FastStripe within your webhook handlers to make additional API calls:

In [ ]:
from faststripe.core import StripeApi

sapi = StripeApi(os.environ['STRIPE_SECRET_KEY'])

def handle_checkout_completed(checkout_session):
    """Enhanced checkout handler using FastStripe"""
    session_id = checkout_session['id']
    
    # Get full session details using FastStripe
    session = sapi.checkout_sessions.fetch(session_id, expand=['customer'])
    
    customer = session.customer
    print(f"Customer details: {customer.name} ({customer.email})")
    
    if session.mode == 'subscription':
        # Get subscription details
        subscription = sapi.subscriptions.fetch(session.subscription)
        print(f"Subscription status: {subscription.status}")
        
        # Update customer metadata
        sapi.customers.update(
            customer.id,
            metadata={
                'subscription_id': subscription.id,
                'plan_name': 'Pro Plan',
                'activated_date': '2024-01-20'
            }
        )
    
    # Send confirmation email, fulfill order, etc.
    print(f"Processing completed for {customer.email}")

## Best practices

- Always verify webhook signatures to ensure they come from Stripe
- Make your webhook handlers idempotent (safe to call multiple times)
- Respond with 200 status code quickly; do heavy processing asynchronously
- Log webhook events for debugging and monitoring
- Use webhook metadata to correlate events with your application data
- Test your webhooks thoroughly using the Stripe CLI
- Set up proper error handling and retry logic for failed webhook processing